# DirAPI Example 1: .txt and .json loader

In this notebook, let us create an API associated with ../data as the first example of `dirapi`.

## Install dirapi

First, install dirapi.
Here, use `-e` option of pip to install dirapi.

In [1]:
# !pip install -e ../../

## Libraries

Import required libraries: some standard libraries and `dirapi`.

In [2]:
import json
import os
from typing import Any, Dict, Iterable

from dirapi import create_api, help_tree

%reload_ext autoreload
%autoreload 2

In [3]:
# const.
ROOT_DIREC = "../data/"

## Tools

Let us define some methods which we will use below.

In [4]:
def load_txt(path: str) -> str:
    """Load text data
    """
    with open(path) as f:
        return f.read()

In [5]:
def load_json(path: str) -> Dict[str, Any]:
    """Load json data
    """
    with open(path) as f:
        return json.load(f)

## Directory structure

Before creating an API associated with `../data/`, let us the structure.

In [6]:
def tree(root: str, only_file: bool=False) -> Iterable[str]:
    for root_, dirs, files in os.walk(root):
        if not only_file:
            yield root_
        for file in files:
            yield os.path.join(root_, file)

In [7]:
list(tree(ROOT_DIREC))

['../data/',
 '../data/jsons',
 '../data/jsons\\sample1.json',
 '../data/jsons\\sample2.json',
 '../data/txts',
 '../data/txts\\test1.txt',
 '../data/txts\\test2.txt']

`../data` has two subdirectories: `jsons` and `txts`.
`jsons` has two json files and `txts` has also two text files.

Their contents are as follows:

In [8]:
for fpath in tree(ROOT_DIREC, True):
    line: str = f"=== Content of {fpath} ==="
    print(line)
    print(load_txt(fpath))
    print("="*len(line))

=== Content of ../data/jsons\sample1.json ===
{
    "name": "Sample1",
    "val": 1
}
=== Content of ../data/jsons\sample2.json ===
{
    "name": "Sample2",
    "val": 2
}
=== Content of ../data/txts\test1.txt ===
This is a test file.
=== Content of ../data/txts\test2.txt ===
This is also a test file.


## Create Simple API

Now, we have already known the structure of `../data`.

Let us start to create a **simple** API associated with `../data`.

This **simple** API provides interfaces to load the contents of json and text files as string.

In [9]:
Api = create_api(ROOT_DIREC, {"load": load_txt})

Using `help_tree`, you can see the structure of an object.

In [10]:
print(help_tree(Api))

Api
Api.Jsons
Api.Jsons.Sample1
Api.Jsons.Sample1.load
Api.Jsons.Sample2
Api.Jsons.Sample2.load
Api.Txts
Api.Txts.Test1
Api.Txts.Test1.load
Api.Txts.Test2
Api.Txts.Test2.load



Good!

`Api` structure corresponds to the structure of `../data`

So, let us use the API!

In [11]:
Api.Jsons.Sample1.load()

'{\n    "name": "Sample1",\n    "val": 1\n}'

In [12]:
Api.Jsons.Sample2.load()

'{\n    "name": "Sample2",\n    "val": 2\n}'

In [13]:
Api.Txts.Test1.load()

'This is a test file.'

In [14]:
Api.Txts.Test2.load()

'This is also a test file.'

If you want to read the docstring of each load method, you can use `__doc__` attribute or `help`.

If you use `help`, you should apply `help` to `func` attribute of method because interfaces in this API partially wrap the original methods with `functools.partial`.

In [15]:
print(Api.Txts.Test2.load.__doc__)

Load text data
    


In [16]:
help(Api.Txts.Test2.load.func)

Help on function load_txt in module __main__:

load_txt(path: str) -> str
    Load text data



## Create more sophisticated API

In the previous section, we created a simple API.

Here, let us create a more sophisticated API associated with `../data`.

This API provides json loader and txt loader.

Using `ext_2_func_map` argument, you can give methods according to the extension.

In [17]:
Api2 = create_api(ROOT_DIREC, ext_2_func_map={".json": {"load": load_json}, ".txt": {"load": load_txt}})

The structure of Api2 is as follows:

In [18]:
print(help_tree(Api2))

Api
Api.Jsons
Api.Jsons.Sample1
Api.Jsons.Sample1.load
Api.Jsons.Sample2
Api.Jsons.Sample2.load
Api.Txts
Api.Txts.Test1
Api.Txts.Test1.load
Api.Txts.Test2
Api.Txts.Test2.load



Of course, the structure of Api2 is same with the structure of Api.

In [19]:
help_tree(Api2) == help_tree(Api)

True

OK.

So let us use the more sophisticated API.

In [20]:
Api2.Jsons.Sample1.load()

{'name': 'Sample1', 'val': 1}

In [21]:
Api2.Jsons.Sample2.load()

{'name': 'Sample2', 'val': 2}

In [22]:
Api2.Txts.Test1.load()

'This is a test file.'

In [23]:
Api2.Txts.Test2.load()

'This is also a test file.'

Great!

We loaded json files as `dict` and text files as `str`.